# 💸 Projeto Final | Sistema de Controle Financeiro

# Etapa 1: Importação das Bibliotecas Necessárias e declaração de Lista para armazenar os registros

In [1]:
import csv
import json
from datetime import datetime
from enum import Enum
from typing import Optional, Union, Literal, Dict

In [2]:
registros = []

class EnumTipoRegistro(Enum):
    RECEITA = 'RECEITA'
    DESPESAS = 'DESPESAS'
    INVESTIMENTOS = 'INVESTIMENTOS'

class EnumFiltragens(Enum):
    DATA = 'DATA'
    TIPO = 'TIPO'
    VALOR = 'VALOR'

# Etapa 2: Definição das Funções Básicas
Função para Criar Novos Registros

In [3]:
def criar_registro(tipo: EnumTipoRegistro, valor: float, registros_atuais: list, data_registro: datetime = datetime.now()) -> None:
    """
        Cria um registro financeiro com base no tipo de transação e valor fornecidos.

        Args:
            tipo (EnumTipoRegistro): O tipo de registro, que pode ser RECEITA, DESPESAS ou INVESTIMENTO.
            valor (float): O valor monetário do registro. Para despesas, o valor será convertido em negativo.
            registros_atuais (list): A lista de registros que será atualizada com o novo registro.
            data_registro (datetime, opcional): A data do registro. Se não for fornecida, a data atual será usada.

        Returns:
            None
    """
    montante: Optional[float] = None
    
    if tipo == EnumTipoRegistro.DESPESAS:
        valor = -abs(valor)
    elif tipo == EnumTipoRegistro.INVESTIMENTOS:
        montante = calcular_rendimento(valor, data_registro) 
    
    registro = {
        'DATA': data_registro.strftime('%Y-%m-%d'),
        'DIA': data_registro.day,
        'MES': data_registro.month,
        'ANO': data_registro.year,
        'TIPO': tipo.value,
        'VALOR': valor,
        'MONTANTE': montante or 'X'
    }

    registros_atuais.append(registro)
    
def calcular_rendimento(capital: float, data_investimento: datetime, taxa: float = 0.01) -> float:
    """
        Calcula o montante de um investimento com base na capital inicial, data de investimento e taxa de rendimento.

        Args:
            capital (float): O valor inicial do investimento.
            data_investimento (datetime): A data em que o investimento foi realizado.
            taxa (float, opcional): A taxa de rendimento diária. O padrão é 0.01 (1%).

        Returns:
            float: O montante acumulado do investimento até a data atual.
    """
    dias = (datetime.now() - data_investimento).days
    montante = capital * (1 + taxa) ** dias
    return montante

Função para Ler Registros

In [4]:


def ler_registros(filtro: EnumFiltragens, valor: Union[str, float, EnumTipoRegistro], registros: list) -> list:
    """
        Filtra uma lista de registros financeiros com base no critério especificado.

        Args:
            filtro (EnumFiltragens): O critério de filtragem (DATA, TIPO, VALOR).
            valor (Union[str, float, EnumTipoRegistro]): O valor a ser comparado para a filtragem.
            registros (list): A lista de registros a ser filtrada.

        Returns:
            list: Uma lista de registros que correspondem ao critério de filtragem.
    """

    def filtrar_por_data():
        return [r for r in registros if r['data'] == valor]

    def filtrar_por_tipo():
        tipo_valor = valor.value if isinstance(valor, EnumTipoRegistro) else valor
        return [r for r in registros if r['tipo'] == tipo_valor]

    def filtrar_por_valor():
        return [r for r in registros if r['valor'] == valor]

    switch = {
        EnumFiltragens.DATA: filtrar_por_data,
        EnumFiltragens.TIPO: filtrar_por_tipo,
        EnumFiltragens.VALOR: filtrar_por_valor
    }
    
    return switch.get(filtro, lambda: registros)()


Função para Atualizar Registros

In [5]:
def atualizar_registro(indice: int, registros: list, tipo: Optional[EnumTipoRegistro] = None, valor: Optional[float] = None) -> None:
  """
    Atualiza um registro na lista de registros financeiros.

    Args:
        indice (int): O índice do registro a ser atualizado.
        registros (list): A lista de registros financeiros.
        tipo (Optional[EnumTipoRegistro]): O novo tipo de transação (opcional).
        valor (Optional[float]): O novo valor da transação (opcional).
  """
  if 0 <= indice < len(registros):
    registro = registros[indice]
    
    if tipo is not None:
      if tipo == EnumTipoRegistro.INVESTIMENTOS:
        registro['MONTANTE'] = calcular_rendimento(valor, datetime.now()) 
      registro[EnumFiltragens.TIPO.value] = tipo.value
    if valor is not None:
      registro[EnumFiltragens.VALOR.value] = valor if tipo != EnumTipoRegistro.DESPESAS else -abs(valor)

    registro[EnumFiltragens.DATA.value] = datetime.now().strftime('%Y-%m-%d')
  else:
    raise IndexError(f"Índice fora do intervalo: {indice}")

Função para Deletar Registros

In [6]:
def deletar_registro(indice: int, registros: list) -> None:
    """
        Deleta um registro da lista de registros financeiros.

        Args:
            indice (int): O índice do registro a ser deletado.
            registros (list): A lista de registros financeiros.
    """
    if 0 <= indice < len(registros):
        del registros[indice]
    else:
        raise IndexError(f"Índice fora do intervalo: {indice}")

# Etapa 3: Funções Adicionais

Função para Atualizar Rendimento

In [7]:
def atualiza_rendimento(registros: list) -> None:
    """
      Atualiza os valores de rendimento para todos os registros de investimento.

      Args:
        registros (list): A lista de registros financeiros.
    """
    for registro in registros:
        if registro['TIPO'] == EnumTipoRegistro.INVESTIMENTOS.value:
            capital = registro['VALOR']
            data_investimento = datetime.strptime(registro['DATA'], '%Y-%m-%d')
            registro['MONTANTE'] = calcular_rendimento(capital, data_investimento)

Função para Exportar Relatório

In [8]:
def exportar_relatorio(formato: Literal['csv', 'json']  = 'csv') -> None:
    """
        Exporta o relatório no formato especificado.

        Args:
            formato (Literal['csv', 'json']): O formato de exportação, 'csv' ou 'json'. O padrão é 'csv'.
    """
    _exportar_csv() if formato == 'csv' else _exportar_json() if formato == 'json' else None

def _exportar_csv() -> None:
    """
        Exporta os registros para um arquivo CSV.
    """
    with open('relatorio.csv', mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=registros[0].keys())
        writer.writeheader()
        writer.writerows(registros)

def _exportar_json() -> None:
    """
        Exporta os registros para um arquivo JSON.
    """
    with open('relatorio.json', 'w') as file:
        json.dump(registros, file, indent=4)

Função de Agrupamento

In [9]:
def agrupar_registros(registros: list, criterio: str) -> dict:
    """
        Agrupa os registros por um critério especificado e calcula o total de valores para cada grupo.

        Args:
            registros (list): A lista de registros financeiros.
            criterio (str): O critério de agrupamento ('TIPO', 'DIA', 'MES', 'ANO').

        Returns:
            dict: Um dicionário com o total de valores para cada grupo.
    """
    agrupamento = {}
    for registro in registros:
        chave = registro.get(criterio)
        if chave is not None:
            if chave not in agrupamento:
                agrupamento[chave] = 0
            agrupamento[chave] += registro['VALOR']
    return agrupamento

# Etapa 4: Testando as Funções

In [10]:
# Criando registros
registros = []
registros2 = []

criar_registro(EnumTipoRegistro.RECEITA, 500, registros)
criar_registro(EnumTipoRegistro.DESPESAS, 100, registros)
criar_registro(EnumTipoRegistro.INVESTIMENTOS, 20, registros)

print(registros)

criar_registro(EnumTipoRegistro.INVESTIMENTOS, 20, registros2, datetime(2024, 8, 2))
print(registros2)


[{'DATA': '2024-08-13', 'DIA': 13, 'MES': 8, 'ANO': 2024, 'TIPO': 'RECEITA', 'VALOR': 500, 'MONTANTE': 'X'}, {'DATA': '2024-08-13', 'DIA': 13, 'MES': 8, 'ANO': 2024, 'TIPO': 'DESPESAS', 'VALOR': -100, 'MONTANTE': 'X'}, {'DATA': '2024-08-13', 'DIA': 13, 'MES': 8, 'ANO': 2024, 'TIPO': 'INVESTIMENTOS', 'VALOR': 20, 'MONTANTE': 20.0}]
[{'DATA': '2024-08-02', 'DIA': 2, 'MES': 8, 'ANO': 2024, 'TIPO': 'INVESTIMENTOS', 'VALOR': 20, 'MONTANTE': 22.31336693330633}]


In [11]:
# Criando registros e exportando para csv e json
registros = []

criar_registro(EnumTipoRegistro.RECEITA, 500, registros)
criar_registro(EnumTipoRegistro.DESPESAS, 100, registros)
criar_registro(EnumTipoRegistro.INVESTIMENTOS, 10000, registros)

exportar_relatorio('csv')
exportar_relatorio('json')

Lendo Registros

In [12]:
# Exemplo de uso
registros = [
  {'data': '2024-08-13', 'tipo': 'RECEITA', 'valor': 1000},
  {'data': '2024-08-13', 'tipo': 'DESPESAS', 'valor': -200},
  {'data': '2024-08-15', 'tipo': 'INVESTIMENTOS', 'valor': 500}
]

# Filtrando por tipo
registrosFiltradosPorValor = ler_registros(EnumFiltragens.VALOR, -200, registros)
registrosFiltradosPorTipo = ler_registros(EnumFiltragens.TIPO, EnumTipoRegistro.RECEITA, registros)
registrosFiltradosPorData = ler_registros(EnumFiltragens.DATA, '2024-08-15', registros)

print(registrosFiltradosPorValor, registrosFiltradosPorTipo, registrosFiltradosPorData, sep='\n')

[{'data': '2024-08-13', 'tipo': 'DESPESAS', 'valor': -200}]
[{'data': '2024-08-13', 'tipo': 'RECEITA', 'valor': 1000}]
[{'data': '2024-08-15', 'tipo': 'INVESTIMENTOS', 'valor': 500}]


Atualizando um Registro e deletando

In [13]:
registros = []
criar_registro(EnumTipoRegistro.DESPESAS, 500, registros, datetime(2024, 8, 10))

print(registros)

atualizar_registro(0, registros, EnumTipoRegistro.INVESTIMENTOS, 1000)

print(registros)

deletar_registro(0, registros)

print(registros)

[{'DATA': '2024-08-10', 'DIA': 10, 'MES': 8, 'ANO': 2024, 'TIPO': 'DESPESAS', 'VALOR': -500, 'MONTANTE': 'X'}]
[{'DATA': '2024-08-13', 'DIA': 10, 'MES': 8, 'ANO': 2024, 'TIPO': 'INVESTIMENTOS', 'VALOR': 1000, 'MONTANTE': 1000.0}]
[]


Agrupando por Tipo

In [14]:
#  Agrupando dados 

registros = []

criar_registro(EnumTipoRegistro.RECEITA, 500, registros, datetime(2024, 1, 10))
criar_registro(EnumTipoRegistro.RECEITA, 500, registros, datetime(2023, 3, 10))
criar_registro(EnumTipoRegistro.DESPESAS, 100, registros, datetime(2023, 3, 10))
criar_registro(EnumTipoRegistro.DESPESAS, 100, registros, datetime(2022, 3, 10))
criar_registro(EnumTipoRegistro.INVESTIMENTOS, 20, registros, datetime(2022, 4, 10))

agrupamento = agrupar_registros(registros, 'TIPO')

print(agrupamento)

{'RECEITA': 1000, 'DESPESAS': -200, 'INVESTIMENTOS': 20}


In [15]:
# atualizações de montantes

registros = [
  {'DATA': '2024-01-01', 'TIPO': 'RECEITA', 'VALOR': 1000.0, 'MONTANTE': 'X'},
  {'DATA': '2024-02-01', 'TIPO': 'DESPESAS', 'VALOR': -500.0, 'MONTANTE': 'X'},
  {'DATA': '2024-03-01', 'TIPO': 'INVESTIMENTOS', 'VALOR': 2000.0, 'MONTANTE': 2100.0},
  {'DATA': '2024-04-01', 'TIPO': 'INVESTIMENTOS', 'VALOR': 1500.0, 'MONTANTE': 1550.0}
]

for registro in registros:
    print(registro) 

atualiza_rendimento(registros)

for registro in registros:
    print(registro) 

{'DATA': '2024-01-01', 'TIPO': 'RECEITA', 'VALOR': 1000.0, 'MONTANTE': 'X'}
{'DATA': '2024-02-01', 'TIPO': 'DESPESAS', 'VALOR': -500.0, 'MONTANTE': 'X'}
{'DATA': '2024-03-01', 'TIPO': 'INVESTIMENTOS', 'VALOR': 2000.0, 'MONTANTE': 2100.0}
{'DATA': '2024-04-01', 'TIPO': 'INVESTIMENTOS', 'VALOR': 1500.0, 'MONTANTE': 1550.0}
{'DATA': '2024-01-01', 'TIPO': 'RECEITA', 'VALOR': 1000.0, 'MONTANTE': 'X'}
{'DATA': '2024-02-01', 'TIPO': 'DESPESAS', 'VALOR': -500.0, 'MONTANTE': 'X'}
{'DATA': '2024-03-01', 'TIPO': 'INVESTIMENTOS', 'VALOR': 2000.0, 'MONTANTE': 10328.961765295999}
{'DATA': '2024-04-01', 'TIPO': 'INVESTIMENTOS', 'VALOR': 1500.0, 'MONTANTE': 5690.564443529742}
